In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, Imputer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import collections

## Train Data!
---

In [2]:
df_train = pd.read_csv('train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_train.drop(columns=['Ticket', 'Name', 'Cabin', 'Embarked'], inplace=True)
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


In [4]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
def check_missing_data(df):
    columns = df.columns
    for i in columns:
        if pd.isna(df[i]).sum(axis = 0) > 0:
            print('Column: {} has {} NaN value'.format(i, pd.isna(df[i]).sum(axis = 0)))
            
check_missing_data(df_train)

In [ ]:
df_train.plot.scatter('Age', 'PassengerId')

In [ ]:
# Fill Values
#df_train['Age'][df_train.Age > 0].mean()
df_train['Age'] = df_train['Age'].fillna(value=df_train['Age'][df_train.Age > 0].mean())
check_missing_data(df_train)

In [ ]:
df_train.describe()

In [ ]:
df_train.head()

In [ ]:
# Forecasters
forecasters_train = df_train.iloc[:, 2:7].values
forecasters_train

In [ ]:
# Classes
classes_train = df_train.iloc[:, 1].values
classes_train

In [ ]:
# LABEL ENCODER
forecasters_label_encoder = LabelEncoder()
forecasters_train[:, 0] = forecasters_label_encoder.fit_transform(forecasters_train[:, 0])
forecasters_train[:, 1] = forecasters_label_encoder.fit_transform(forecasters_train[:, 1])
forecasters_train[:, 3] = forecasters_label_encoder.fit_transform(forecasters_train[:, 3])
forecasters_train[:, 4] = forecasters_label_encoder.fit_transform(forecasters_train[:, 4])
forecasters_train

In [ ]:
# # ONE HOT ENCODER
# one_hot_encoder = OneHotEncoder(categorical_features=[1, 2, 4, 5])
# forecasters_train = one_hot_encoder.fit_transform(forecasters_train).toarray()
# forecasters_train

In [ ]:
# SCALING
scaler = StandardScaler()
forecasters_train = scaler.fit_transform(forecasters_train)
forecasters_train

## Test Data!
---

In [ ]:
df_test = pd.read_csv('test.csv')
df_test.head()

In [ ]:
df_test_class = pd.read_csv('gender_submission.csv')
df_test_class.head()

In [ ]:
df_test.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked'], inplace=True)
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
check_missing_data(df_test)

In [ ]:
df_test.plot.scatter('Fare', 'PassengerId')

In [ ]:
df_test.plot.scatter('Age', 'PassengerId')

In [ ]:
#df_test['Age'] = df_test['Age'].fillna(value=df_test['Age'][df_test.Age > 0].mean())
df_test['Age'] = df_test['Age'].fillna(value=df_test['Age'][df_test.Age > 0].mean())
#df_test['Age'][df_test.Age > 0].mean()
df_test['Fare'] = df_test['Fare'].fillna(value=df_test['Fare'][df_test.Age < 500].mean())
check_missing_data(df_test)

In [ ]:
df_test.head()

In [ ]:
forecasters_test = df_test.iloc[:, 1:6].values
forecasters_test

In [ ]:
classes_test= df_test_class.iloc[:, 1].values
classes_test

In [ ]:
# LABEL ENCODER
forecasters_test_label_encoder = LabelEncoder()
forecasters_test[:, 0] = forecasters_test_label_encoder.fit_transform(forecasters_test[:, 0])
forecasters_test[:, 1] = forecasters_test_label_encoder.fit_transform(forecasters_test[:, 1])
forecasters_test[:, 3] = forecasters_test_label_encoder.fit_transform(forecasters_test[:, 3])
forecasters_test[:, 4] = forecasters_test_label_encoder.fit_transform(forecasters_test[:, 4])
forecasters_test

In [ ]:
# # ONE HOT ENCODER
# one_hot_encoder_test = OneHotEncoder(categorical_features=[1, 2, 4, 5])
# forecasters_test = one_hot_encoder_test.fit_transform(forecasters_test).toarray()
# forecasters_test

In [ ]:
# SCALING
scaler = StandardScaler()
forecasters_test = scaler.fit_transform(forecasters_test)
forecasters_test

In [ ]:
forecasters_test

In [ ]:
classes_test

In [ ]:
estimator = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
estimator.fit(forecasters_train, classes_train)
predictions = estimator.predict(forecasters_test)

In [ ]:
count = collections.Counter(classes_test)
print('Line Base Classifier {:.2f}%'.format(count[0]/(count[0]+count[1])*100))

In [ ]:
precision = accuracy_score(classes_test, predictions)
print('Precision: {:.2f}%'.format(precision * 100))

In [ ]:
matrix = confusion_matrix(classes_test, predictions)
print('Confusion Matrix:'
  '\n\t0\t1\n0:\t{}\t{}'
  '\n1:\t{}\t{}'.format(matrix[0][0], matrix[0][1], matrix[1][0], matrix[1][1]))

In [ ]:
np.savetxt("predictions.csv", predictions, fmt="%d", delimiter=",")